In [17]:
#必要ライブラリの読み込み
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms, models
from torch.utils.data import DataLoader
from torch.optim import Adam
import os
from PIL import Image
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import joblib
from PIL import Image
import torchvision.transforms as T
from torch.utils.data import Dataset
import torch.optim as optim


In [21]:
# 1. モデルを読み込む
model_d = joblib.load('bananaDetect3.pkl')
model_c = joblib.load('bananaClassify4.pkl')

# デバイスの指定 (GPUが利用可能な場合はGPUを使用し、それ以外の場合はCPUを使用)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [28]:

# 2. 画像の前処理(切り取り前)
def preprocess_image(image):
    transform_d = T.Compose([
        T.ToTensor(),  # 画像をテンソルに変換
    ])
    return transform_d(image).unsqueeze(0)  # バッチ次元を追加




# 3. モデルに画像を渡してバウンディングボックスを検出
def detect_bounding_box(model, image_tensor, threshold=0.5):
    model.eval()
    with torch.no_grad():
        image_tensor = image_tensor.to(device)
        predictions = model(image_tensor)

    # しきい値を超える最もスコアの高いバウンディングボックスを見つける
    best_score = 0.0
    best_box = None
    for score, label, box in zip(predictions[0]['scores'], predictions[0]['labels'], predictions[0]['boxes']):
        if score > threshold and score > best_score:
            best_score = score
            best_box = box

    return best_box



#4. 画像の前処理を追加（切り取り後）
def preprocess_cropped_image(image):
    transform = T.Compose([
        T.Resize((224, 224)),  # 画像を指定のサイズにリサイズ
        T.RandomHorizontalFlip(),  # ランダムな水平反転
        T.RandomRotation(10),  # ランダムな回転（最大10度）
#         T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),  # 色情報を変更
        T.ToTensor(),  # 画像をテンソルに変換
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # 画像を標準化
    ])
    return transform(image).unsqueeze(0)  # バッチ次元を追加


# 5. 切り取った画像に対して分類を行う
def classify_banana(model, image):
    # 画像の前処理を適用
#     image_tensor = preprocess_image(image)
    
    #ここで、切り取り画像の前処理を行う。
    
    # モデルに画像を入力し、予測を取得
    with torch.no_grad():
        outputs = model(image)
    
    # 予測クラスの取得
    _, predicted_class = outputs.max(1)
    
    return predicted_class.item()





# 画像フォルダ内の各画像に対して処理
image_folder = 'bananas_images_test'  # 画像フォルダのパスを指定
filename='test.jpg';

image_path = os.path.join(image_folder, filename)
# 画像を開く
pil_image = Image.open(image_path)
# バウンディングボックスを検出
image_tensor = preprocess_image(pil_image)
bounding_box = detect_bounding_box(model_d, image_tensor)
        
if bounding_box is not None:
    # バウンディングボックス情報を取得
    x1, y1, x2, y2 = map(int, bounding_box.tolist())
    cropped_image = pil_image.crop((x1, y1, x2, y2))
            
    # 切り取り画像に前処理を適用
    cropped_image_tensor = preprocess_cropped_image(cropped_image)
            
            
    # 分類を実行
    predicted_class = classify_banana(model_c, cropped_image_tensor)
            
    # 予測クラスを表示
    print(f"Predicted class for {filename}: {predicted_class}")


Predicted class for test.jpg: 0
